In [ ]:
import os
from tqdm import tqdm

path = '/share/jproject/fg538/r-006-gpu-4/data'
train_count = 0
val_count = 0

for folder in tqdm(os.listdir('{}/train2/'.format(path))):
    train_count += len(os.listdir('{}/train2/{}/'.format(path, folder)))
for folder in tqdm(os.listdir('{}/keras_data/val_dir/'.format(path))):
    val_count += len(os.listdir('{}/keras_data/val_dir/{}/'.format(path, folder)))
train_count, val_count

In [ ]:
import keras
from keras.models import load_model
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 299, 299
train_data_dir = "/share/jproject/fg538/r-006-gpu-4/data/train2/"
validation_data_dir = "/share/jproject/fg538/r-006-gpu-4/data/keras_data/val_dir/"
batch_size = 32
epochs = 2
nb_classes = 14951

train_datagen = ImageDataGenerator(rotation_range=20, 
                                   shear_range=.2, 
                                   horizontal_flip=True,
                                   zca_whitening=True,
                                   fill_mode='reflect')
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(train_data_dir, 
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size)
val_generator = val_datagen.flow_from_directory(validation_data_dir, 
                                                target_size=(img_width, img_height), 
                                                batch_size=batch_size)

model = applications.inception_v3.InceptionV3(weights="imagenet", 
                                              include_top=False, 
                                              input_shape = (img_width, img_height, 3))
#model.summary()

#Adding custom Layer 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(nb_classes, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", 
                    optimizer = optimizers.SGD(lr=0.0001, momentum=0.9),
                    metrics=["accuracy"])

# callbacks
# Save the model according to the conditions 
checkpoint = ModelCheckpoint("inception_v3_3.h5", 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             period=1)
#early = EarlyStopping(monitor='val_acc', min_delta=0, patience=2, verbose=1, mode='auto')
tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', 
                                          histogram_freq=1, 
                                          batch_size=batch_size, 
                                          write_graph=True,
                                          write_grads=True)

# Train the model 
history = model_final.fit_generator(train_generator, 
                                    steps_per_epoch=38283, 
                                    epochs=epochs, 
                                    verbose=1,
                                    callbacks=[checkpoint],
                                    validation_data=val_generator,
                                    validation_steps=2500,
                                    class_weight='auto')

/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/keras/preprocessing/image.py:492: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 1224092 images belonging to 14951 classes.
Found 81574 images belonging to 100 classes.
Epoch 1/2


/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/keras/preprocessing/image.py:571: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/keras/preprocessing/image.py:589: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


28394/38283 [=====================>........] - ETA: 2:17:24 - loss: 5.3468 - acc: 0.3005